In [1]:
import nltk
from nltk.corpus import wordnet_ic
import numpy as np
import nltk.tokenize
from nltk.corpus import wordnet as wn
import scipy.spatial.distance as distance
import gensim

# Information content for similarities
nltk.download('wordnet_ic')
# Wordnet
nltk.download('wordnet')
# Multilingual wordnet
nltk.download('omw')

c:\python34\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[nltk_data] Downloading package wordnet_ic to
[nltk_data]     C:\Users\Jukka\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jukka\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to
[nltk_data]     C:\Users\Jukka\AppData\Roaming\nltk_data...
[nltk_data]   Package omw is already up-to-date!


True

Tutorial based on http://www.nltk.org/howto/wordnet.html

## Words
We can query words/lemmas with wn.synsets("word")

We can optionally pass arguments as <b>"word.pos.nn"</b> to query a specific lemma

Also possible to translate meanings to different languages

Can also use 

wn.synsets('dog', pos=wn.VERB)

In [2]:
# All meanings for word cool
print(wn.synsets('cool'))
# Most frequent meaning and translations
print("============")
print(wn.synset('cool.n.01').definition())
print(wn.synset('cool.n.01').lemma_names('eng'))
print(wn.synset('cool.n.01').lemma_names('fin'))
print("============")
# We can also map back to english!
print(wn.synsets('viileys', lang='fin'))

[Synset('cool.n.01'), Synset('aplomb.n.01'), Synset('cool.v.01'), Synset('cool.v.02'), Synset('cool.v.03'), Synset('cool.a.01'), Synset('cool.s.02'), Synset('cool.a.03'), Synset('cool.a.04'), Synset('cool.s.05'), Synset('cool.s.06')]
the quality of being at a refreshingly low temperature
['cool']
['viileys', 'vilpoisuus']
[Synset('coldness.n.02'), Synset('aplomb.n.01'), Synset('coldness.n.03'), Synset('chilliness.n.01'), Synset('cool.n.01'), Synset('distance.n.04'), Synset('withdrawal.n.04')]


## Synsets

A set of words that share a common meaning

Each synset has one or more lemmas

Lemmas in synset then contains 
* Gloss - dictionary like definition
* Examples - Examples of word usage

In [3]:
# Function to explore word synsets
# Input a word, optional language and Noun (n), Verb(v), Adj(a)
def print_word_synsets(word, l='eng', p=""):
    print('Count of near synonyms: ' + str(len(wn.synsets(word, lang=l, pos=p))))
    for syn in wn.synsets(word, lang=l, pos=p):
        print("====================================================")
        print("Synset: " + syn.name())
        print("Gloss: " + wn.synset(syn.name()).definition())
        print("examples: " + str(len(wn.synset(syn.name()).examples())))
        for example in wn.synset(syn.name()).examples():
            print("\t" + example)

In [4]:
print_word_synsets('ice', l='eng', p="v")

Count of near synonyms: 3
Synset: frost.v.01
Gloss: decorate with frosting
examples: 1
	frost a cake
Synset: ice.v.02
Gloss: cause to become ice or icy
examples: 1
	an iced summer drink
Synset: ice.v.03
Gloss: put ice on or put on ice
examples: 1
	Ice your sprained limbs


## Wordnet structure, relations

We can travel the defined relations in the graph

In [5]:
def print_noun_relations(synset):
    print("Lemma: \t" + str(synset) + "\n")
    print("hypernyms: \t"+ str(synset.hypernyms()))
    print("hyponyms: \t"+str(synset.hyponyms()))
    print("member_mero: \t"+str(synset.member_meronyms()))
    print("member_holo: \t"+str(synset.member_holonyms()))
    print("Antonyms: \t"+str(synset.lemmas()[0].antonyms()))

def print_verb_relations(synset):
    print("Lemma: \t" + str(synset) + "\n")
    print("hypernyms: \t"+ str(synset.hypernyms()))
    print("Troponym: \t"+str(synset.hyponyms()))
    print("Entails: \t"+str(synset.entailments()))
    print("Antonyms: \t"+str(synset.lemmas()[0].antonyms()))
    
# Travel the relations
pizza = wn.synset("pizza.n.01")
print_noun_relations(pizza)
print("\n")
diving = wn.synset("dive.v.01")
print_verb_relations(diving)

Lemma: 	Synset('pizza.n.01')

hypernyms: 	[Synset('dish.n.02')]
hyponyms: 	[Synset('anchovy_pizza.n.01'), Synset('cheese_pizza.n.01'), Synset('pepperoni_pizza.n.01'), Synset('sausage_pizza.n.01'), Synset('sicilian_pizza.n.01')]
member_mero: 	[]
member_holo: 	[]
Antonyms: 	[]


Lemma: 	Synset('dive.v.01')

hypernyms: 	[Synset('descend.v.01')]
Troponym: 	[Synset('chute.v.01'), Synset('crash-dive.v.01'), Synset('duck.v.02'), Synset('nosedive.v.01'), Synset('power-dive.v.01')]
Entails: 	[]
Antonyms: 	[]


## Similarities

We can measure word similarities on wordnet

In wordnet similarity means that a word can be replaced by another one always

Word similarity can mean <b>similarity</b> or <b>words being related </b>
* Similar <-> tractor, car
* Related but not similar car <-> gasoline

Words are defined to be similar if
* they share meaning(s) 
* are near synonyms 


In [6]:
dog = wn.synset('dog.n.01')
cat = wn.synset('cat.n.01')

# Path similarity

Path similarity is calculated by how many edges there are between two words

for $pathlen(a,b)$ we pick the edge with least distance

Every edge is defined to have the same weight

$sim(a,b) = \frac{1}{pathlen(a,b)}$

In [7]:
dog.path_similarity(cat)

0.2

## Information content

IC tries to fix that every edge has the same weight

We can use information content to determine similarities between words. Always based on probability of finding a word in a corpus

In NLTK we can download ready made IC-dictionaries or use our own.


## Resnik similarity

Similarity based on LCS, that refers to lowest common subsumer. e.g. (word with lowest distance from two concepts)
* cat, dog have hypernym of animal and mammal
* mammal is lower common hypernym

$P(c) =  \frac{\sum count(w)}{N}$

$ LCS(a, b) = $ first node that's hypernym for $a,b$ 

$sim(a,b)_{resnik} = -log( P( LCS(a, b ) )$


In [8]:
# Load and use ic-libraries
brown_ic = wordnet_ic.ic('ic-brown.dat')
semcor_ic = wordnet_ic.ic('ic-semcor.dat')
print(dog.res_similarity(cat, brown_ic))
print(dog.res_similarity(cat, semcor_ic))

7.911666509036577
7.2549003421277245


## Lin-similarity

Another way to calculate similarities based on information contents (based on resniks method)

$sim(a,b) = \frac{2 log P(LCS(a,b))}{log P(a) + log P(b)}$

In [9]:
print(dog.lin_similarity(cat, brown_ic))
print(dog.lin_similarity(cat, semcor_ic))

0.8768009843733973
0.8863288628086228


## Worthwile to mention

* Myrphs -> we can try to lemmatize the word
* Domains of a word 

In [10]:
# Extras
print(wn.morphy('churches'))
print(wn.synset('code.n.03').topic_domains())

church
[Synset('computer_science.n.01')]


## Lesk algorithm

Tries to decode a word meaning based on the surrounding words.

We loop over all the senses of the word and choose one that shares most words with the target

simplified lesk algorithm in the book didnt seem to include hyponyms, but it seems to work better with those (as we get more descriptions for a word)

<b> Is similarity based on glosses </b>

In [11]:
# Calculate how many words in query sentance agree with examples
def overlapcontext(sense, sentence):
    sentence = nltk.word_tokenize(sentence)
    gloss = set(nltk.word_tokenize(sense.definition()))
    for expmle in sense.examples():
         gloss.union(nltk.word_tokenize(expmle))
    return len(gloss.intersection(sentence))

# Compare senses of a word and pick one that has most overlapping words
def lesk(word, sentence):
    # Would be also fine to assign most frequent sense here
    bestsense = None
    maxoverlap = 0
    # Try to lemmatize the word with morphy
    word = wn.morphy(word) if wn.morphy(word) is not None else word
    # Loop all the possible meanings of the word
    for sense in wn.synsets(word):
        overlap = overlapcontext(sense, sentence)
        # Subtypes might be related to word!
        for h in sense.hyponyms():
            overlap = overlap + overlapcontext(h, sentence)
        if overlap > maxoverlap:
                maxoverlap = overlap
                bestsense = sense
    return bestsense

In [12]:
sense = lesk("cone", "A traffic cone was tipped over")
print(str(sense) + " | " + str(sense.definition()))
print(str(sense.hyponyms()))

Synset('cone.n.03') | cone-shaped mass of ovule- or spore-bearing scales or bracts
[Synset('fir_cone.n.01'), Synset('galbulus.n.01'), Synset('pinecone.n.01')]


## Lesk algorithm with W2V similarity measure

Instead of counting the overlap, we can pick the meaning that is most similar to the query sentance 

In [13]:
# Load google's w2v
word2vec = gensim.models.KeyedVectors.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True, limit=200000) 
word_vectors = word2vec.wv

c:\python34\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
# Calculate w2v cosine sim between two words
def calculate_w2v_cossim(sentance1, sentance2):
    doc1 = [word_vectors[word] for word in sentance1 if word in word2vec.vocab]
    doc2 = [word_vectors[word] for word in sentance2 if word in word2vec.vocab]
    
    doc1 = np.mean(doc1, axis=0)
    doc2 = np.mean(doc2, axis=0)
    return 1 - distance.cosine(doc1, doc2)

# loop over all examples of a sense and pick the most similar one
def overlap_w2v_context(sense, sentence):
    sentence = nltk.word_tokenize(sentence)
    gloss = nltk.word_tokenize(sense.definition())
    
    similarities = []
    similarities.append(calculate_w2v_cossim(sentence, gloss))
    for expmle in sense.examples():
        similarities.append(calculate_w2v_cossim(sentence, nltk.word_tokenize(expmle)))
    return np.max(similarities)
    
# w2v - Lesk algorithm!
def lesk_w2v(word, sentence):
    bestsense = None
    maxsimilarity = 0
    word = wn.morphy(word) if wn.morphy(word) is not None else word
    # Loop all the possible meanings of the word
    for sense in wn.synsets(word):
        tmp = []
        tmp.append(overlap_w2v_context(sense, sentence))
        for h in sense.hyponyms():
            tmp.append(overlap_w2v_context(h, sentence)) 
        # Only pick most similar entry for use, so we dont favor meanings with many examples
        similarity = np.max(tmp)
        if similarity > maxsimilarity:
                maxsimilarity = similarity
                bestsense = sense
    return bestsense

# Get outcomes for traditional and W2V lesk algorithms
def test_lesks(word, sentance):
    sense_w2v = lesk_w2v(word, sentance)
    sense_trad = lesk(word, sentance)
    
    print("======== Traditional =======")
    print(sense_trad)
    print(sense_trad.definition())
    print("=========== V2W ============")
    print(sense_w2v)
    print(sense_w2v.definition())

In [15]:
test_lesks("cook", "i like to cook food")

======== Traditional =======
Synset('cook.v.03')
transform and make suitable for consumption by heating
=========== V2W ============
Synset('cook.n.01')
someone who cooks food


In [16]:
test_lesks("cone", "there are many pine cones in the trees")

======== Traditional =======
Synset('cone.n.03')
cone-shaped mass of ovule- or spore-bearing scales or bracts
=========== V2W ============
Synset('cone.n.03')
cone-shaped mass of ovule- or spore-bearing scales or bracts


In [18]:
import xml.etree.ElementTree as ET
# Read and prep XML
tree = ET.parse("eng-coarse-all-words.xml")
root = tree.getroot()
texts = root.findall('text')
keyDictionary = {}

# read the keys!
with open("fs_baseline.key.txt") as f:
    for line in f.readlines():
        tmp = line.split()
        # 1: id present with the data, 2: Key of the item in wordnet
        keyDictionary[tmp[1]] = tmp[2]

right_traditional = 0
right_w2v = 0
counter = 0
# Loop over the text and instances
for text in texts:
    # One text contains many sentace - elements
    for sentance in text.findall('sentence'):
        text = " ".join(sentance.itertext()).replace('\n','')
        # We want to find meaning for words behind <instance> tags
        for instance in sentance.findall('instance'):
            instance_id = instance.get('id')
            lemma_traditional = lesk(instance.text, text)
            lemma_w2v = lesk_w2v(instance.text, text)
            # See if we had returned a meaning, and if the keys matched
            if lemma_traditional:
                # All keys distinct and im lazy -> this should be fine
                for lemma in lemma_traditional.lemmas():
                    if lemma.key() == keyDictionary[instance_id]:
                        right_traditional += 1

            if lemma_w2v:
                for lemma in lemma_w2v.lemmas():
                    if lemma.key() == keyDictionary[instance_id]:
                        right_w2v += 1
    
print("Traditional algorithm got right: " + str(right_traditional/len(keyDictionary)))
print("\n")
print("W2V algorithm got right: " + str(right_w2v/len(keyDictionary)))


c:\python34\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\python34\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\python34\lib\site-packages\numpy\core\_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


Traditional algorithm got right: 0.376817981489643


W2V algorithm got right: 0.35654473336271486
